# File Name Checking

In [4]:
Year = 202408

Location = r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(Year)+"\\Finance"
File = "Debtors Listing and Customer Balance Report as at Aug 2024"

str(File[-8:])

'Aug 2024'

In [5]:
str(File[-4:])

'2024'

In [6]:
BalanceOS = "Balance_31stAugust2024"
BalanceOS

'Balance_31stAugust2024'

# ---------------------------   _Debtor Listing_ ---------------------------

In [8]:
#Year = "202406"
#Location = r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(Year)+"\\Finance"

#BalanceOS = "Balance_30thJune2024"
#File = "Debtors Listing and Customer Balance Report as at Jun 2024"

#Year = "202405"
#BalanceOS = "Balance_31stMay2024"
#File = "Debtors Listing and Customer Balance Report as at May 2024"
#Location = r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(Year)+"\\Final\Finance"


#control + F + dropdown new to search bar to find and replace 
import pandas as pd
import numpy as np
import xlsxwriter

#warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

Isl_Cost = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Debtors Listing Islamic (Cost)', header=5) # skiprows = 2, nrows=18,  usecols= 'A:D'
Isl_Profit = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Debtors Listing Islamic (Profit', header=3) # skiprows = 2, nrows=18,  usecols= 'A:D'
Mora = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Modification MORA & R&R '+str(File[-8:-5])+str(File[-4:]), header=5) # skiprows = 2, nrows=18,  usecols= 'A:D'
Conv = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Debtors Listing (C) '+str(File[-8:]), header=2) # skiprows = 2, nrows=18,  usecols= 'A:D'
Accrued = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Accrued Interest '+str(File[-8:]), header=4) # skiprows = 2, nrows=18,  usecols= 'A:D'
Others_conv = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Other Debtors Conv '+str(File[-8:]), header=4) # skiprows = 2, nrows=18,  usecols= 'A:D'
Others_Isl = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Other Debtors Islamic '+str(File[-8:]), header=4) # skiprows = 2, nrows=18,  usecols= 'A:D'

IIS = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='IIS '+str(File[-8:]), header=4) # skiprows = 2, nrows=18,  usecols= 'A:D'
PIS = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='PIS '+str(File[-8:]), header=4) # skiprows = 2, nrows=18,  usecols= 'A:D'

MRate = pd.read_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(Year)+"\\Working\\00. Month End Rate.xlsx",sheet_name="Sheet1", header=2) # skiprows = 2, nrows=18,  usecols= 'A:D'

LDB_prev = pd.read_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(Year-1)+"\\Loan Database as Jul 2024 (python) - Final.xlsx", sheet_name='Export Worksheet', header=1) # skiprows = 2, nrows=18,  usecols= 'A:D'


In [47]:
#LDB_prev = pd.read_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - SPD\Loan Database\LBD 202412 - 202401 (CSP Sector).xlsx", sheet_name="Apr24", header=0) # skiprows = 2, nrows=18,  usecols= 'A:D'

#Contigent = pd.read_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\202405\Contigent Liability MAY2024 (python).xlsx", header=0) # skiprows = 2, nrows=18,  usecols= 'A:D'


#Isl_Cost
#Isl_Profit
#Mora
#Conv
#Accrued
#Others_conv
#Others_Isl

In [9]:
#-------------------------------------------------------------islamic----------------------------------------------------------

#Debtors Listing Islamic (Cost)
Isl_Cost1 = Isl_Cost.iloc[np.where(~Isl_Cost['Customer\nAccount'].isna())]

Isl_Cost1.columns = Isl_Cost1.columns.str.replace("\n", "_")
Isl_Cost1.columns = Isl_Cost1.columns.str.replace(" ", "")

Isl_Cost1.Customer_Account = Isl_Cost1.Customer_Account.astype(int)
Isl_Cost1.Disbursement = Isl_Cost1.Disbursement.astype(float)
Isl_Cost1.Cost_Payment = Isl_Cost1.Cost_Payment.astype(float)
Isl_Cost1[BalanceOS] = Isl_Cost1[BalanceOS].astype(float)

Isl_Cost1.rename(columns={"Disbursement":"Disbursement - old"}, inplace=True)
Isl_Cost1['Disbursement'] = Isl_Cost1['Disbursement - old'].fillna(0) + Isl_Cost1['Adjustment/_Capitalisation'].fillna(0)

Isl_Cost1.rename(columns={"Cost_Payment":"Cost_Payment - old"}, inplace=True)
Isl_Cost1['Cost_Payment'] = Isl_Cost1['Cost_Payment - old'].fillna(0) - Isl_Cost1['Adjustment/_Capitalisation.1'].fillna(0)

Isl_Cost2 = Isl_Cost1.fillna(0).groupby(['Company','Customer_Account'\
,'Currency'])[['Disbursement'\
,'Cost_Payment',BalanceOS]].sum().reset_index()

Isl_Cost2 = Isl_Cost2.rename(columns={BalanceOS: 'Principal'}).fillna(0).sort_values(by=['Principal'],ascending=[True])
#Isl_Cost2['Sheet'] = 'Debtors Listing Islamic (Cost)'
Isl_Cost2['Financing_Type'] = 'Islamic'

C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\2117793227.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Isl_Cost1.Customer_Account = Isl_Cost1.Customer_Account.astype(int)
C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\2117793227.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Isl_Cost1.Disbursement = Isl_Cost1.Disbursement.astype(float)
C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\2117793227.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [11]:
#Debtors Listing Islamic (Profit)
Isl_Profit1 = Isl_Profit.iloc[np.where(~Isl_Profit['Customer\nAccount'].isna())]

Isl_Profit1.columns = Isl_Profit1.columns.str.replace("\n", "_")
Isl_Profit1.columns = Isl_Profit1.columns.str.replace(" ", "")

Isl_Profit1.Customer_Account = Isl_Profit1.Customer_Account.astype(int)
Isl_Profit1.Unearned_Profit = Isl_Profit1.Unearned_Profit.astype(float)
Isl_Profit1.Profit_Payment = Isl_Profit1.Profit_Payment.astype(float)
Isl_Profit1[BalanceOS] = Isl_Profit1[BalanceOS].astype(float)

Isl_Profit1.rename(columns={"Profit_Payment":"Profit_Payment - old"}, inplace=True)
Isl_Profit1['Profit_Payment'] = Isl_Profit1['Profit_Payment - old'].fillna(0) - Isl_Profit1['Adjustment/_Capitalisation.1'].fillna(0)

Isl_Profit2 = Isl_Profit1.fillna(0).groupby(['Company','Customer_Account'\
,'Currency'])[['Unearned_Profit','Rental(Ijarah)','Profit_Payment',BalanceOS]].sum().reset_index()

Isl_Profit2 = Isl_Profit2.rename(columns={BalanceOS: 'Interest'}).fillna(0).sort_values(by=['Interest'],ascending=[True])
#Isl_Profit2['Sheet'] = 'Debtors Listing Islamic (Profit)'
Isl_Profit2['Financing_Type'] = 'Islamic'

C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\3831447813.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Isl_Profit1.Customer_Account = Isl_Profit1.Customer_Account.astype(int)
C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\3831447813.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Isl_Profit1.Unearned_Profit = Isl_Profit1.Unearned_Profit.astype(float)
C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\3831447813.py:9: SettingWithCopyWarning: 
A value is trying to be se

In [12]:
#Combine Islamic Cost+Profit
A001 = Isl_Cost2.merge(Isl_Profit2,on=['Customer_Account','Company','Currency','Financing_Type'],how='outer',indicator=True)

A001 = A001.drop(columns=['_merge'])

A002 = A001.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type'])[['Disbursement'\
,'Cost_Payment','Principal','Unearned_Profit','Rental(Ijarah)','Profit_Payment','Interest']].sum().reset_index()

NamaCompany = A001[['Company','Customer_Account']].drop_duplicates('Customer_Account', keep='first')
A003 = A002.merge(NamaCompany,on='Customer_Account',how='left')

In [13]:
#Modification MORA & R&R Apr2024
Mora1 = Mora.fillna(0).rename(columns={'Borrower code': 'Customer_Account',
                            'Borrower':'Company',
                            'Modification impact (RM)':'Mora',
                            'Islamic/ conventional':'Financing_Type'}).iloc[np.where((~Mora['Borrower code'].isna())&(Mora['Borrower code']!='Borrower code'))]

Mora1.columns = Mora1.columns.str.replace("\n", "_")
Mora1.columns = Mora1.columns.str.replace(" ", "")

Mora1.Customer_Account = Mora1.Customer_Account.astype(int)
Mora1.Mora = Mora1.Mora.astype(float)

Mora1.loc[Mora1.Currency.isin(['RM']),'Currency'] = 'MYR'

A004 = A003.merge(Mora1[['Customer_Account','Company','Currency','Financing_Type','SLOacceptancedate','Mora']],on=['Customer_Account','Company','Currency','Financing_Type'],how='outer',indicator=True)

NamaMora = A004[['Company','Customer_Account']].drop_duplicates('Customer_Account', keep='first')
A004 = A004.drop(['Company','_merge'],axis=1).merge(NamaMora,on='Customer_Account',how='left')

A004 = A004.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Cost_Payment','Principal','Unearned_Profit','Rental(Ijarah)','Profit_Payment','Interest','Mora']].sum().reset_index()

In [14]:
#Other Debtors Islamic Apr2024
Others_Isl1 = Others_Isl.iloc[np.where(~Others_Isl.Customer.isna())].fillna(0)

Others_Isl1.columns = Others_Isl1.columns.str.replace("\n", "_")
Others_Isl1.columns = Others_Isl1.columns.str.replace(" ", "")

Others_Isl1.rename(columns={'Customer': 'Customer_Account',
                            'SearchTerm':'Company',
                            'Crcy':'Currency',
                            'Accumulatedbalance':'Other_Charges'},inplace=True)

Others_Isl1.Customer_Account = Others_Isl1.Customer_Account.astype(int)
Others_Isl1.Other_Charges = Others_Isl1.Other_Charges.astype(float)

Others_Isl1 = Others_Isl1.fillna(0).groupby(['Company','Customer_Account'])[['Other_Charges']].sum().reset_index()

Others_Isl1['Financing_Type'] = 'Islamic'

A005 = A004.merge(Others_Isl1,on=['Customer_Account','Company','Financing_Type'],how='outer',indicator=True)

NamaOther = A005[['Company','Customer_Account','Currency']].drop_duplicates('Customer_Account', keep='first')
A005 = A005.drop(['Company','Currency','_merge'],axis=1).merge(NamaOther,on='Customer_Account',how='left')

A005 = A005.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Cost_Payment','Principal','Unearned_Profit','Rental(Ijarah)','Profit_Payment','Interest','Mora','Other_Charges']].sum().reset_index()


C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\4274878480.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Others_Isl1 = Others_Isl.iloc[np.where(~Others_Isl.Customer.isna())].fillna(0)


In [15]:
#PIS
PIS1 = PIS.iloc[np.where(~PIS.Customer.isna())].fillna(0)

PIS1.columns = PIS1.columns.str.replace("\n", "_")
PIS1.columns = PIS1.columns.str.replace(" ", "")

PIS1.rename(columns={'Customer': 'Customer_Account',
                            'SearchTerm':'Company',
                            'Crcy':'Currency',
                            'Accumulatedbalance':'Interest_in_Suspense'},inplace=True)

PIS1.Customer_Account = PIS1.Customer_Account.astype(int)
PIS1.Interest_in_Suspense = PIS1.Interest_in_Suspense.astype(float)

PIS1 = PIS1.fillna(0).groupby(['Company','Customer_Account'])[['Interest_in_Suspense']].sum().reset_index()

PIS1['Financing_Type'] = 'Islamic'

A006 = A005.merge(PIS1,on=['Customer_Account','Company','Financing_Type'],how='outer',indicator=True)

NamaPIS = A006[['Company','Customer_Account','Currency']].drop_duplicates('Customer_Account', keep='first')
A006 = A006.drop(['Company','Currency','_merge'],axis=1).merge(NamaPIS,on='Customer_Account',how='left')

A006 = A006.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Cost_Payment','Principal','Unearned_Profit','Rental(Ijarah)','Profit_Payment','Interest','Mora','Other_Charges','Interest_in_Suspense']].sum().reset_index()


C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\754960510.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  PIS1 = PIS.iloc[np.where(~PIS.Customer.isna())].fillna(0)


In [16]:
#-------------------------------------------------conv--------------------------------------------------

#Debtors Listing Conv Apr 2024
Conv1 = Conv.iloc[np.where(~Conv['Customer Account Number'].isna())]

Conv1.columns = Conv1.columns.str.replace("\n", "_")
Conv1.columns = Conv1.columns.str.replace(" ", "")

Conv1 = Conv1.rename(columns={'CustomerAccountNumber': 'Customer_Account',
                            'CustomerName':'Company',
                            'LoanCurrency':'Currency',
                            'ClosingPrincipal':'Principal'}).fillna(0)

Conv1.Customer_Account = Conv1.Customer_Account.astype(int)
Conv1.Principal = Conv1.Principal.astype(float)

#[['Customer_Account','Company','Currency','Disbursement','Repayment','Principal']]

Conv1.rename(columns={"Disbursement":"Disbursement - old"}, inplace=True)
Conv1['Disbursement'] = Conv1['Disbursement - old'].fillna(0) + Conv1['AdjustmentCapitalization'].fillna(0)

Conv1.rename(columns={"Repayment":"Repayment - old"}, inplace=True)
Conv1['Repayment'] = Conv1['Repayment - old'].fillna(0) - Conv1['AdjustmentCapitalization.1'].fillna(0)

Conv1 = Conv1.fillna(0).groupby(['Company','Customer_Account'\
,'Currency'])[['Disbursement'\
,'Repayment','Principal']].sum().reset_index()

Conv1['Financing_Type'] = 'Conventional'

C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\2099751802.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  'ClosingPrincipal':'Principal'}).fillna(0)


In [17]:
#Accrued Interest Apr2024
Accrued1 = Accrued.iloc[np.where(~Accrued.Customer.isna())].fillna(0)

Accrued1.columns = Accrued1.columns.str.replace("\n", "_")
Accrued1.columns = Accrued1.columns.str.replace(" ", "")

Accrued1.rename(columns={'Customer': 'Customer_Account',
                            'SearchTerm':'Company',
                            'Crcy':'Currency',
                            'Accumulatedbalance':'Interest',
                            'Debitrept.period':'Interest_For_the_Month',
                            'Creditreportper.':'Profit_Payment'},inplace=True)

Accrued1.Customer_Account = Accrued1.Customer_Account.astype(int)
Accrued1.Interest = Accrued1.Interest.astype(float)
Accrued1.Interest_For_the_Month = Accrued1.Interest_For_the_Month.astype(float)
Accrued1.Profit_Payment = Accrued1.Profit_Payment.astype(float)

Accrued1.loc[(Accrued1['SGLInd.'].isin(['X'])),'Interest_For_the_Month'] = Accrued1.Interest_For_the_Month
Accrued1.loc[(~Accrued1['SGLInd.'].isin(['X'])),'Interest_For_the_Month'] = 0

Accrued1.loc[(Accrued1['SGLInd.'].isin(['X'])),'Profit_Payment'] = Accrued1.Profit_Payment
Accrued1.loc[(~Accrued1['SGLInd.'].isin(['X'])),'Profit_Payment'] = 0

Accrued1 = Accrued1.fillna(0).groupby(['Company','Customer_Account'])[['Interest_For_the_Month','Interest','Profit_Payment']].sum().reset_index()

Accrued1['Financing_Type'] = 'Conventional'

C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\2186211233.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Accrued1 = Accrued.iloc[np.where(~Accrued.Customer.isna())].fillna(0)


In [19]:
#Combine Conv Principal+Accrued
C001 = Conv1.merge(Accrued1,on=['Customer_Account','Company','Financing_Type'],how='outer',indicator=True)

NamaConv = C001[['Company','Customer_Account','Currency']].drop_duplicates('Customer_Account', keep='first')
C002 = C001.drop(['Company','Currency','_merge'],axis=1).merge(NamaConv,on='Customer_Account',how='left')

C002 = C002.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Repayment','Principal','Interest_For_the_Month','Interest','Profit_Payment']].sum().reset_index()

In [21]:
#Other Debtors Apr2024
Others_conv1 = Others_conv.iloc[np.where(~Others_conv.Customer.isna())].fillna(0)

Others_conv1.columns = Others_conv1.columns.str.replace("\n", "_")
Others_conv1.columns = Others_conv1.columns.str.replace(" ", "")

Others_conv1.rename(columns={'Customer': 'Customer_Account',
                            'SearchTerm':'Company',
                            'Crcy':'Currency',
                            'Accumulatedbalance':'Other_Charges'},inplace=True)

Others_conv1.Customer_Account = Others_conv1.Customer_Account.astype(int)
Others_conv1.Other_Charges = Others_conv1.Other_Charges.astype(float)

Others_conv1 = Others_conv1.fillna(0).groupby(['Company','Customer_Account'])[['Other_Charges']].sum().reset_index()

Others_conv1['Financing_Type'] = 'Conventional'

C003 = C002.merge(Others_conv1,on=['Customer_Account','Company','Financing_Type'],how='outer',indicator=True)

NamaOtherConv = C003[['Company','Customer_Account','Currency']].drop_duplicates('Customer_Account', keep='first')
C004 = C003.drop(['Company','Currency','_merge'],axis=1).merge(NamaOtherConv,on='Customer_Account',how='left')

C004 = C004.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Repayment','Principal','Interest_For_the_Month','Interest','Profit_Payment','Other_Charges']].sum().reset_index()


C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\935440056.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Others_conv1 = Others_conv.iloc[np.where(~Others_conv.Customer.isna())].fillna(0)


In [22]:
#IIS
IIS1 = IIS.iloc[np.where(~IIS.Customer.isna())].fillna(0)

IIS1.columns = IIS1.columns.str.replace("\n", "_")
IIS1.columns = IIS1.columns.str.replace(" ", "")

IIS1.rename(columns={'Customer': 'Customer_Account',
                            'SearchTerm':'Company',
                            'Crcy':'Currency',
                            'Accumulatedbalance':'Interest_in_Suspense'},inplace=True)

IIS1.Customer_Account = IIS1.Customer_Account.astype(int)
IIS1.Interest_in_Suspense = IIS1.Interest_in_Suspense.astype(float)

IIS1 = IIS1.fillna(0).groupby(['Company','Customer_Account'])[['Interest_in_Suspense']].sum().reset_index()

IIS1['Financing_Type'] = 'Conventional'

C005 = C004.merge(IIS1,on=['Customer_Account','Company','Financing_Type'],how='outer', indicator=True)

NamaIIS = C005[['Company','Customer_Account','Currency']].drop_duplicates('Customer_Account', keep='first')
C005 = C005.drop(['Company','Currency','_merge'],axis=1).merge(NamaIIS,on='Customer_Account',how='left')

C005 = C005.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Repayment','Principal','Interest_For_the_Month','Interest','Profit_Payment','Other_Charges','Interest_in_Suspense']].sum().reset_index()

C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_25768\760218338.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  IIS1 = IIS.iloc[np.where(~IIS.Customer.isna())].fillna(0)


In [45]:
#-------------------------------------------------combine-------------------------------------------------

C005['Cost_Payment'] = 0
C005['Unearned_Profit'] = C005['Interest_For_the_Month']
#C005['Profit_Payment'] = 0
C005['Mora'] = 0
C005['Rental(Ijarah)'] = 0

C006 = C005[['Customer_Account','Currency','Financing_Type','Company','Disbursement','Repayment','Cost_Payment',
             'Principal','Unearned_Profit','Rental(Ijarah)','Profit_Payment','Interest','Mora','Other_Charges',
             'Interest_in_Suspense']]

A006['Repayment'] = 0

A007 = A006[['Customer_Account','Currency','Financing_Type','Company','Disbursement','Repayment','Cost_Payment',
             'Principal','Unearned_Profit','Rental(Ijarah)','Profit_Payment','Interest','Mora','Other_Charges',
             'Interest_in_Suspense']]

#Isl_Cost1.columns = Isl_Profit1.columns = Mora1.columns = Conv1.columns
#appendR = pd.concat([Isl_Cost1,Isl_Profit1,Mora1,Conv1] )

C006.columns = A007.columns
appendR = pd.concat([C006,A007])

NamaappendR = appendR.iloc[np.where(~(appendR.Currency.isin([0,'0'])))][['Company','Currency','Customer_Account']].drop_duplicates('Customer_Account', keep='first')
appendR = appendR.drop(['Company','Currency'],axis=1).merge(NamaappendR,on='Customer_Account',how='left')

appendfinal = appendR.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Repayment','Cost_Payment','Principal','Unearned_Profit','Rental(Ijarah)','Profit_Payment','Interest','Mora','Other_Charges','Interest_in_Suspense']].sum().reset_index()

appendfinal['Total Loans Outstanding (MYR)'] = appendfinal['Principal'] + appendfinal['Interest'] + appendfinal['Mora'] + appendfinal['Other_Charges']


In [46]:
#appendfinal.iloc[np.where(appendfinal['Customer_Account']==500749)] #500642 500640 500749

In [58]:
#appendfinal['Customer_Account'].value_counts()

In [48]:
appendfinal['Cost Payment/Principal Repayment (MYR)'] = (-1*appendfinal['Repayment']) + appendfinal['Cost_Payment']
appendfinal['Accrued Profit/Interest of the month (MYR)'] = appendfinal['Unearned_Profit'] + appendfinal['Rental(Ijarah)'] #+ profit for the month

appendfinal.rename(columns={'Customer_Account':'Finance(SAP) Number',
                           'Currency':'Facility Currency',
                           'Financing_Type':'Type of Financing',
                            "Company":"Customer Name",
                            "Disbursement":"Disbursement/Drawdown (MYR)",
                           'Principal':'Cost/Principal Outstanding (MYR)',
                           'Profit_Payment':"Profit Payment/Interest Repayment (MYR)",
                           'Interest':'Cumulative Accrued Profit/Interest (MYR)',
                           'Mora':'Modification of Loss (MYR)',
                           'Other_Charges':'Other Charges (MYR)',
                           'Interest_in_Suspense':'Income/Interest in Suspense (MYR)'}, inplace=True)

appendfinal.drop(columns=['Repayment','Cost_Payment','Unearned_Profit','Rental(Ijarah)'], axis=1, inplace=True)

                           #"Repayment":"1. Cost Payment/Principal Repayment (MYR)",
                           #"Cost_Payment":'2. Cost Payment/Principal Repayment (MYR)',


                           #'Unearned_Profit':'Accrued Profit/Interest of the month (MYR)',
                           #'Rental(Ijarah)':'Ijarah',

In [54]:
C006.head(1)

,Customer_Account,Currency,Financing_Type,Company,Disbursement,Repayment,Cost_Payment,Principal,Unearned_Profit,Rental(Ijarah),Profit_Payment,Interest,Mora,Other_Charges,Interest_in_Suspense
0,500010,0,Conventional,PT BUMI,0.0,0.0,0,1.21e+07,0.0,0,0.0,0.0,0,228850.2,5.29e+06


In [57]:
C006.shape

(44, 15)

In [59]:
#C006.iloc[np.where(C006['Currency']==0)]

In [60]:
#appendfinal.head(1)
#appendfinal.iloc[np.where(appendfinal['Facility Currency'].isin([0]))]

In [62]:
#LDB_prev.head(1)

In [63]:
#LDB_prev[''].value_counts()

#Cumulative Disbursement/Drawdown (Facility Currency)
#Cumulative Disbursement/Drawdown (MYR)
#Cumulative Cost Payment/Principal Repayment (Facility Currency)
#Cumulative Cost Payment/Principal Repayment (MYR)
#Cumulative Profit Payment/Interest Repayment (Facility Currency)
#Cumulative Profit Payment/Interest Repayment (MYR)

In [66]:
LDB_prev.head(1)

,No.,CIF Number,EXIM Account No.,Application System Code,CCRIS Master Account Number,CCRIS Sub Account Number,Finance(SAP) Number,Company Group,Customer Name,Relationship Manager (RM),Banking Team,Ownership,Officer in Charge,Syndicated / Club Deal,Nature of Account,Facility,Facility Currency,Type of Financing,Shariah Contract / Concept,Fund Type,Incentive,Programme,Guarantee,Status,Post Approval Stage,Date of Ready for Utilization (RU),Restructured / Rescheduled(Y/N),Amount Approved / Facility Limit (Facility Currency),Amount Approved / Facility Limit (MYR),Cost/Principal Outstanding (Facility Currency),Cost/Principal Outstanding (MYR),Contingent Liability Letter of Credit (Facility Currency),Contingent Liability Letter of Credit (MYR),Contingent Liability (Facility Currency),Contingent Liability (MYR),Account Receivables/Past Due Claims (Facility Currency),Account Receivable/Past Due Claims (MYR),Total Banking Exposure (Facility Currency),Total Banking Exposure (MYR),Accrued Profit/Interest of the month (Facility Currency),Accrued Profit/Interest of the month (MYR),Modification of Loss (Facility Currency),Modification of Loss (MYR),Cumulative Accrued Profit/Interest (Facility Currency),Cumulative Accrued Profit/Interest (MYR),Penalty/Ta`widh (Facility Currency),Penalty/Ta`widh (MYR),Income/Interest in Suspense (Facility Currency),Income/Interest in Suspense (MYR),Other Charges (Facility Currency),Other Charges (MYR),Total Loans Outstanding (Facility Currency),Total Loans Outstanding (MYR),Expected Credit Loss (ECL) LAF (Facility Currency),Expected Credit Loss LAF (ECL) (MYR),Expected Credit Loss C&C (ECL) (Facility Currency),Expected Credit Loss C&C (ECL) (MYR),Disbursement/Drawdown Status,Unutilised/Undrawn Amount (Facility Currency),Unutilised/Undrawn Amount (MYR),Disbursement/Drawdown (Facility Currency),Disbursement/Drawdown (MYR),Cumulative Disbursement/Drawdown (Facility Currency),Cumulative Disbursement/Drawdown (MYR),Cost Payment/Principal Repayment (Facility Currency),Cost Payment/Principal Repayment (MYR),Cumulative Cost Payment/Principal Repayment (Facility Currency),Cumulative Cost Payment/Principal Repayment (MYR),Profit Payment/Interest Repayment (Facility Currency),Profit Payment/Interest Repayment (MYR),Cumulative Profit Payment/Interest Repayment (Facility Currency),Cumulative Profit Payment/Interest Repayment (MYR),Ta`widh Payment/Penalty Repayment (Facility Currency),Ta`widh Payment/Penalty Repayment (MYR),Cumulative Ta`widh Payment/Penalty Repayment (Facility Currency),Cumulative Ta`widh Payment/Penalty Repayment (MYR),Other Charges Payment (Facility Currency),Other Charges Payment (MYR),Cumulative Other Charges Payment (Facility Currency),Cumulative Other Charges Payment (MYR),Rating at Origination,PD,PF,CRMS Obligor Risk Rating,CRMS CG Rating,PD (%),LGD (%),Risk Analyst,MFRS9 Staging,BNM Main Sector,BNM Sub Sector,EXIM Main Sector,Industry (Risk),Industry Classification,Purpose of Financing,Date Approved at Origination,Approval Authority,LO issuance Date,Date of LO Acceptance,1st Disbursement/Drawdown Date,1st Payment/Repayment Date,Expiry of Availability Period,Facility Agreement Date,Annual Review Date,Watchlist Review Date,Maturity/Expired Date,Grace Period (Month),Moratorium Period (Month),Start Moratorium Date,Tenure (Month),Payment/Repayment Frequency (Profit/Interest),Payment/Repayment Frequency (Cost/Principal),Effective cost of borrowings,Profit/Interest Margin,Average Profit/Interest Rate,Ta`widh Compensation/Penalty Rate,Operation Country,Country Exposure,Country Rating,Region,Market Type,Classification of Entity / Customer Type,Entity / Customer Type,Classification of Residency Status,Residency Status,Corporate Type,SME Commercial Corporate,Corporate Status,Justification on Corporate Status,Restructured / Rescheduled,Date of Approval Restructured / Rescheduled,Effective Date,Reason,Date Untagged from R&R,Justification for Untagged,Frequency,Date of Overdue,Overdue (Days),Month in Arrears,Over

In [74]:
appendfinal['Finance(SAP) Number'] = appendfinal['Finance(SAP) Number'].astype(str)

LDB_prev['Finance(SAP) Number'] = LDB_prev['Finance(SAP) Number'].astype(str)

LDB_prev.columns = LDB_prev.columns.str.replace("\n", "")

appendfinal_ldb = appendfinal.merge(LDB_prev.iloc[np.where(LDB_prev['EXIM Account No.']!="Total")][['EXIM Account No.','Finance(SAP) Number',
                                              'Facility Currency',
                                              'Cumulative Disbursement/Drawdown (Facility Currency)',
                                              'Cumulative Disbursement/Drawdown (MYR)',
                                              'Cumulative Cost Payment/Principal Repayment (Facility Currency)',
                                              'Cumulative Cost Payment/Principal Repayment (MYR)',
                                              'Cumulative Profit Payment/Interest Repayment (Facility Currency)',
                                              'Cumulative Profit Payment/Interest Repayment (MYR)']].drop_duplicates('Finance(SAP) Number',keep='first'),on=['Finance(SAP) Number'],how='left', suffixes=('_x', ''),indicator=True)

appendfinal_ldb['Facility Currency'] = appendfinal_ldb['Facility Currency'].astype(str)
appendfinal_ldb['Facility Currency'] = appendfinal_ldb['Facility Currency'].str.strip()

appendfinal2 = appendfinal_ldb.merge(MRate[['Month','Curr']].rename(columns={'Month':'Facility Currency'}), on='Facility Currency', how='left')

appendfinal2['Cost/Principal Outstanding (Facility Currency)'] = appendfinal2['Cost/Principal Outstanding (MYR)']/appendfinal2['Curr']
appendfinal2['Accrued Profit/Interest of the month (Facility Currency)'] = appendfinal2['Accrued Profit/Interest of the month (MYR)']/appendfinal2['Curr']
appendfinal2['Modification of Loss (Facility Currency)'] = appendfinal2['Modification of Loss (MYR)']/appendfinal2['Curr']
appendfinal2['Cumulative Accrued Profit/Interest (Facility Currency)'] = appendfinal2['Cumulative Accrued Profit/Interest (MYR)']/appendfinal2['Curr']
appendfinal2['Income/Interest in Suspense (Facility Currency)'] = appendfinal2['Income/Interest in Suspense (MYR)']/appendfinal2['Curr']
appendfinal2['Other Charges (Facility Currency)'] = appendfinal2['Other Charges (MYR)']/appendfinal2['Curr']
appendfinal2['Total Loans Outstanding (Facility Currency)'] = appendfinal2['Total Loans Outstanding (MYR)']/appendfinal2['Curr']
appendfinal2['Disbursement/Drawdown (Facility Currency)'] = appendfinal2['Disbursement/Drawdown (MYR)']/appendfinal2['Curr']
appendfinal2['Cost Payment/Principal Repayment (Facility Currency)'] = appendfinal2['Cost Payment/Principal Repayment (MYR)']/appendfinal2['Curr']
appendfinal2['Profit Payment/Interest Repayment (Facility Currency)'] = appendfinal2['Profit Payment/Interest Repayment (MYR)']/appendfinal2['Curr']


appendfinal2['Cumulative Disbursement/Drawdown (Facility Currency) New'] = appendfinal2['Disbursement/Drawdown (Facility Currency)'] +  appendfinal2['Cumulative Disbursement/Drawdown (Facility Currency)'] 
appendfinal2['Cumulative Disbursement/Drawdown (MYR) New'] = appendfinal2['Disbursement/Drawdown (MYR)'] +  appendfinal2['Cumulative Disbursement/Drawdown (MYR)'] 
appendfinal2['Cumulative Cost Payment/Principal Repayment (Facility Currency) New'] = appendfinal2['Cost Payment/Principal Repayment (Facility Currency)'] +  appendfinal2['Cumulative Cost Payment/Principal Repayment (Facility Currency)'] 
appendfinal2['Cumulative Cost Payment/Principal Repayment (MYR) New'] = appendfinal2['Cost Payment/Principal Repayment (MYR)'] +  appendfinal2['Cumulative Cost Payment/Principal Repayment (MYR)'] 
appendfinal2['Cumulative Profit Payment/Interest Repayment (Facility Currency) New'] = appendfinal2['Profit Payment/Interest Repayment (Facility Currency)'] +  appendfinal2['Cumulative Profit Payment/Interest Repayment (Facility Currency)'] 
appendfinal2['Cumulative Profit Payment/Interest Repayment (MYR) New'] = appendfinal2['Profit Payment/Interest Repayment (MYR)'] +  appendfinal2['Cumulative Profit Payment/Interest Repayment (MYR)'] 

appendfinal2.sort_values('Total Loans Outstanding (MYR)', ascending=False, inplace=True)

In [76]:
#LDB_prev['EXIM Account No.'].value_counts()

In [75]:
appendfinal2['Finance(SAP) Number'].value_counts()

Finance(SAP) Number
500731    1
500913    1
501146    1
500529    1
500838    1
         ..
501018    1
500558    1
500879    1
500557    1
500937    1
Name: count, Length: 275, dtype: int64

In [73]:
#appendfinal2.shape
#appendfinal2.iloc[np.where(appendfinal2['Finance(SAP) Number']=="500725")] #500642 500640 500749


In [77]:
appendfinal3 = appendfinal2[['EXIM Account No.','Finance(SAP) Number',
                             'Customer Name',
                             'Facility Currency',
                             'Type of Financing',
                             'Cost/Principal Outstanding (Facility Currency)',
                             'Cost/Principal Outstanding (MYR)',
                             'Accrued Profit/Interest of the month (Facility Currency)',
                             'Accrued Profit/Interest of the month (MYR)',
                             'Modification of Loss (Facility Currency)',
                             'Modification of Loss (MYR)',
                            'Cumulative Accrued Profit/Interest (Facility Currency)',
                            'Cumulative Accrued Profit/Interest (MYR)', 
                             'Income/Interest in Suspense (Facility Currency)',
                             'Income/Interest in Suspense (MYR)',
                             'Other Charges (Facility Currency)',
                             'Other Charges (MYR)',
                             'Total Loans Outstanding (Facility Currency)',
                             'Total Loans Outstanding (MYR)',
                            #'Disbursement/Drawdown (Facility Currency)',
                            #'Disbursement/Drawdown (MYR)',
                             #'Cumulative Disbursement/Drawdown (Facility Currency) New',
                             #'Cumulative Disbursement/Drawdown (Facility Currency)',
                             #'Cumulative Disbursement/Drawdown (MYR) New',
                             #'Cumulative Disbursement/Drawdown (MYR)',
                            #'Cost Payment/Principal Repayment (Facility Currency)',
                            #'Cost Payment/Principal Repayment (MYR)',
                             #'Cumulative Cost Payment/Principal Repayment (Facility Currency) New',
                             #'Cumulative Cost Payment/Principal Repayment (Facility Currency)',
                             #'Cumulative Cost Payment/Principal Repayment (MYR) New',
                             #'Cumulative Cost Payment/Principal Repayment (MYR)',
                            #'Profit Payment/Interest Repayment (Facility Currency)',
                            #'Profit Payment/Interest Repayment (MYR)',
                             #'Cumulative Profit Payment/Interest Repayment (Facility Currency) New',
                             #'Cumulative Profit Payment/Interest Repayment (Facility Currency)',
                             #'Cumulative Profit Payment/Interest Repayment (MYR) New',
                             #'Cumulative Profit Payment/Interest Repayment (MYR)',
                             'Curr']]

In [78]:
appendfinal3.shape

(275, 20)

In [79]:
sum(appendfinal3['Total Loans Outstanding (MYR)'])

4187069880.940652

In [80]:
appendfinal3['Finance(SAP) Number'].value_counts().head(20)

Finance(SAP) Number
500731    1
500913    1
501146    1
500529    1
500838    1
500926    1
500719    1
500725    1
500528    1
500907    1
500779    1
500991    1
500737    1
500559    1
500922    1
500688    1
501035    1
500965    1
500798    1
501075    1
Name: count, dtype: int64

# ---------------------------   _Extract_ ---------------------------

In [82]:
#appendfinal3.to_excel(str(Location[:-7])+'\\01. Debtor Listing '+str(File[-8:])+' (python).xlsx', sheet_name='Export Worksheet', index = False)
appendfinal3.to_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(Year)+"\\Working\\01. Debtor Listing "+str(File[-8:])+" (python).xlsx", sheet_name='Export Worksheet', index = False)

In [84]:
#sum(appendfinal3.iloc[np.where(appendfinal3['Finance(SAP) Number']=="501051")]['Profit Payment/Interest Repayment (Facility Currency)'])

In [ ]:
sum(appendfinal3.iloc[np.where(appendfinal3['Finance(SAP) Number']=="501051")]['Profit Payment/Interest Repayment (MYR)'])

In [ ]:


.format('${:>10,.0f}')

In [7]:
import pandas as pd

x = pd.DataFrame([[5.75,7.32],[1000000,-2]])

styles = {
    'font-family': 'monospace',
    'white-space': 'pre'
}

x_style = x.style.set_properties(**styles)

x_style.format(lambda f: '${:>10}'.format(('({:,.0f})' if f < 0 else '{:,.0f}').format(f)))

,0,1
0,$ 6,$ 7
1,"$ 1,000,000",$ (-2)


In [8]:
sum(x[1])

5.32

In [ ]:
#-------------------------------------------------Contigent-------------------------------------------------

#Contigent1 = Contigent.fillna(0).groupby(['CIF Number','EXIM Account No.','Finance(SAP) Number',\
#                                          'Customer Name','Type of Financing'])[['Contingent Liability (MYR)','Contingent Liability Letter of Credit (MYR)']].sum().reset_index()

#appendfinal_conti = appendfinal.merge(LDB_prev1[['CIF Number','Finance(SAP) Number']].drop_duplicates(),on=['Finance(SAP) Number'],how='left',suffixes=('', '_LDB'),indicator=True)



In [ ]:
appendfinal_conti.shape

In [ ]:
appendfinal.shape

In [ ]:
appendfinal_conti.iloc[np.where(appendfinal_conti._merge!="both")].head(3)

In [ ]:
conti_det['CIF Number'].value_counts()

In [ ]:
conti_det.iloc[np.where(conti_det['CIF Number']=="EXIM000210")]

In [ ]:
append_conti = appendfinal.merge(Contigent1,on=['Customer_Account','Company','Financing_Type'],how='outer',indicator=True)


In [ ]:
#-------------------------------------------------LDB-------------------------------------------------

#LDB_prev1 = LDB_prev.iloc[np.where(~LDB_prev['No.'].isna())]

#LDB_prev1.columns = LDB_prev1.columns.str.replace("\n", "_")
#LDB_prev1.columns = LDB_prev1.columns.str.replace(" ", "")
#LDB_prev1['Finance(SAP) Number'] = LDB_prev1['Finance(SAP) Number'].astype(str)
#appendfinal['Finance(SAP) Number'] = appendfinal['Finance(SAP) Number'].astype(str)

#append_ldb = appendfinal.merge(LDB_prev1[['CIF Number','EXIM Account No.','Finance(SAP) Number','Customer Name','Type of Financing','Facility Currency']],on=['Finance(SAP) Number'],how='left',suffixes=('', '_LDB'),indicator=True)


In [ ]:
#append_ldb['Finance(SAP) Number'].value_counts()

In [ ]:
#append_ldb.iloc[np.where(append_ldb['Finance(SAP) Number']=="500776")]

In [ ]:
#-------------------------------------------------Month Rate-------------------------------------------------

MRate

In [ ]:
#smpi currency lain & insert contigent


In [ ]:
#appendfinal.to_excel(str(Location)+'\\Debtor Listing '+str(File[-12:-4])+' (python).xlsx', sheet_name='Export Worksheet', index = False)


In [ ]:
#writer2 = pd.ExcelWriter(r'C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\Automation LDB\Data Source\202404\1. ECL\Finance\working\\Debtor Listing (python).xlsx',engine='xlsxwriter')

#appendfinal.to_excel(writer2, sheet_name='Export Worksheet', index = False)

#writer2._save()

In [ ]:
print(C005.shape)
print(A006.shape)

In [ ]:
C005.head(1)

In [ ]:
A006.head(1)

In [ ]:
A006.Customer_Account.value_counts()

In [ ]:
appendfinal.shape

In [ ]:
appendfinal.iloc[np.where(appendfinal.Customer_Account==500640)]

In [ ]:
appendfinal.Customer_Account.value_counts()

In [ ]:
#checking v1
sum(appendfinal['Total Loans Outstanding (MYR)'])

In [ ]:
#checking v2
sum(appendfinal['Total Loans Outstanding (MYR)'])